In [1]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from tools.standard_name_utils import optimize_parameters
from tools.common_utils import highlight_diff, read_jsonl

model_path = "../../train-result/2024-06-19/checkpoint-5000"
top_p=0.7 
temperature=0.95

/home/reymond/anaconda3/envs/afac2024/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).cuda()
model = model.eval()

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
Loading checkpoint shards: 100%|██████████| 6/6 [11:00<00:00, 110.15s/it]


In [3]:
json_file_path = 'data/test_b.json'
data_list = read_jsonl(json_file_path)

df = pd.DataFrame(data_list)
df.head()

,conversations
0,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
1,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
2,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
3,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
4,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."


In [4]:
from tools.standard_name_utils import get_standard_api, optimize_api_chain, optimize_parameters

standard_api = get_standard_api("./data/api_定义.json")

data_stock = pd.read_excel('./raw_data/标准名.xlsx',sheet_name='股票标准名')
data_fund = pd.read_excel('./raw_data/标准名.xlsx',sheet_name='基金标准名')
stock_standard_name = data_stock['标准股票名称'].to_list()
fund_standard_name = data_fund['标准基金名称'].to_list()

In [5]:
stock_standard_name[0:5]

['万马股份', '汉王科技', '珠江啤酒', '联络互动', '奇正藏药']

In [6]:
fund_standard_name[0:5]

['华夏成长证券投资基金',
 '华夏大盘精选证券投资基金A类',
 '易方达天天理财货币市场基金R类',
 '华夏优势增长混合型证券投资基金',
 '泰达宏利信用合利定期开放债券型证券投资基金A类']

In [7]:
total_eval_count = 0

with open('data/submit.txt','w', encoding="utf-8") as n:
    for index, row in df.iterrows():    
        single_user_row = row["conversations"][0]
        single_assis_row = row["conversations"][1]
    
        input = single_user_row["content"]
        output = single_assis_row["content"]
        
        response, history = model.chat(tokenizer=tokenizer, query=input, history=[], top_p=top_p, temperature=temperature)
        
        optimized_resp = optimize_parameters(response, fund_standard_name, stock_standard_name)
        
        total_eval_count += 1
        n.write(optimized_resp+'\n')
        if total_eval_count % 10 == 0:
            print("现在是第" + f"{total_eval_count}" + "条数据")

Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
